In [6]:
import networkx as nx
import numpy as np
import torch
from datetime import datetime

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt

import pandas as pd
import torch_geometric.transforms as T

from create_dataset import CreateDataset

from models import GNNModel

import sys
sys.path.append("../FastCover/")
from utils import *

In [7]:
PATH_SAVE_TRAINS_CHECKPOINTS = 'runs/checkpoints/'
PATH_SAVE_TRAINS = 'runs/'

num_features = 1
num_classes  = 2

threshold = 0.5

optimizer_name = "Adam"
lr = 1e-3
epochs = 2

SEED = 13

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layers = ["GCN", "GAT","GraphConv"]

Models = [GNNModel(c_in = 1, c_hidden = 100, c_out = 2, num_layers = 2, layer_name = layer_name, dp_rate=0.1) for 
         layer_name in layers]

In [8]:
def getGraph(n, p = 0.5):
    while True:
        G = nx.erdos_renyi_graph(n, p, directed = False)
        AllConected = True if len([i for i in nx.connected_components(G)]) == 1 else False
        if AllConected:
            break
    return G

def Convert2DataSet(Graphs):
    g = []
    for G in Graphs:
        # Cambiar aquí la forma de cómo calcular dichos valores
        OptimalSet, _ = MDH(G, threshold, print_= False)

        NumNodes = G.number_of_nodes()
        labels = np.zeros(NumNodes)
        labels[OptimalSet] = 1

        dataset = CreateDataset(G, labels)
        data = dataset[0]

        data =  data.to(device)
        g.append(data)
        
    return g

In [9]:
n = 30
p = 0.6
Train = [getGraph(n+i, p) for i in range(10)]

In [ ]:
## Falta por incluir estax

In [21]:
from torch_geometric.nn import GCNConv, GINConv
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.nn import global_mean_pool, global_add_pool

class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, dim_h):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(dataset.num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.lin1 = Linear(dim_h*3, dim_h*3)
        self.lin2 = Linear(dim_h*3, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Node embeddings 
        h1 = self.conv1(x, edge_index)
        h2 = self.conv2(h1, edge_index)
        h3 = self.conv3(h2, edge_index)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

In [22]:
gin = GIN(dim_h=1)
Graphs_Train[0]

NameError: name 'dataset' is not defined

## Train

In [14]:
Graphs_Train = Convert2DataSet(Train)

In [9]:
def train(model, optimizer, data):
        model.train()
        optimizer.zero_grad()

        F.nll_loss(model(data.x, data.edge_index), data.y).backward()
        optimizer.step()
        return model, optimizer
      
    
@torch.no_grad()
def test(data, model):
  model.eval()
  logits = model(data.x, data.edge_index)
  pred = logits.max(1)[1]
  acc = pred.eq(data.y).sum().item() / data.num_nodes
  return acc

In [29]:
torch.manual_seed(SEED)
for i in range(len(Models)):
    print()
    print(f" ----- Model:{layers[i]} -----")
    optimizer = getattr(torch.optim, optimizer_name)(Models[i].parameters(), lr = lr)

    for epoch in range(1, epochs):
        
        for data in Graphs_Train:
            train(Models[i], optimizer, data)
        
        if epoch%5 == 0:
            torch.save(Models[i].state_dict(), f=f"{PATH_SAVE_TRAINS_CHECKPOINTS}Checkpoint-model-{layers[i]}-epoch-{epoch}.pt")
            print(f"Epoch {epoch} saved for {layers[i]}.\n")
        
            Acc = []

            for data in Graphs_Train:
                Acc.append(test(data, Models[i]))
            print(f"Mean Acc: {np.mean(Acc)}")
            print()
        
    dt_string = datetime.now().strftime("%m-%d_%H-%M")
    torch.save(Models[i].state_dict(), f=f"{PATH_SAVE_TRAINS}{layers[i]}_seed_{SEED}_thr_{int(threshold*10)}_date_{dt_string}.pt")
    
    print(f"{layers[i]} saved in {PATH_SAVE_TRAINS}\n")


 ----- Model:GCN -----
Epoch 5 saved for GCN.

Mean Acc: 0.3484966659387413

Epoch 10 saved for GCN.

Mean Acc: 0.3484966659387413

GCN saved in runs/


 ----- Model:GAT -----
Epoch 5 saved for GAT.

Mean Acc: 0.6515033340612588

Epoch 10 saved for GAT.

Mean Acc: 0.6515033340612588

GAT saved in runs/


 ----- Model:GraphConv -----
Epoch 5 saved for GraphConv.

Mean Acc: 0.3484966659387413

Epoch 10 saved for GraphConv.

Mean Acc: 0.3484966659387413

GraphConv saved in runs/



## Test

In [ ]:
PATH_SAVE_TRAINS = 'runs/'

In [ ]:
Test =  [getGraph(n+2*i, p) for i in range(5)]
Graphs_Test = Convert2DataSet(Test)

In [158]:
Acc = []
    for data in Graphs_Test:
        Acc.append(test(data, Models[i]))
    
    records

0.7
0.65625
0.6470588235294118
0.6666666666666666
0.6578947368421053


In [2]:
graph

NameError: name 'graph' is not defined